Train model with any length of the text

In [ ]:
# import library
import pandas as pd
import numpy as np
import torch
from json_to_csv import create_csv

Create csv dataset from json file

In [ ]:
create_csv()

get vectors and vocab from word2vec model

In [ ]:
# model = Word2Vec.load("./updated_word2vec_model.model")
# vocab = model.wv.key_to_index
# vectors = model.wv.vectors


In [ ]:
# model = api.load('word2vec-google-news-300', return_path = False)
# vectors = model.vectors[:2000000]
# vocab = {x:model.key_to_index[x] for x in model.key_to_index if model.key_to_index[x]<2000000}

In [ ]:
# len(vocab)


## Multiclass Text Classification

In [ ]:
# loading the data
data_df = pd.read_csv("./dataset/data.csv")
print(data_df.shape)
data_df.head()


In [ ]:
data_df['intent'].nunique()

In [ ]:
data_df["pattern"]


Data Aug

In [ ]:
from augmentation import data_aug

In [ ]:
data_df["intent"].value_counts()


In [ ]:
data_df = data_aug(data_df)
data_df = data_aug(data_df)
data_df.drop_duplicates(inplace=True)


In [ ]:
data_df.shape


In [ ]:
data_df["intent"].value_counts()


In [ ]:
from preprocessor import Preprocessor


In [ ]:
max_word = 256
preprocessor = Preprocessor()


In [ ]:
data_df = preprocessor.text_encode(data_df, max_word)


In [ ]:
data_df = preprocessor.label_encoding(data_df)


In [ ]:
import pickle
with open("./saved_objects/preprocessor.pkl", "wb") as handle:
    pickle.dump(preprocessor, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
data_df.head()


In [ ]:
max1 = 0
for x in data_df["pattern encoded"].values:
    if max(x[0]) > max1:
        max1 = max(x[0])


In [ ]:
max1


Train val split

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
# split data set
X = list(data_df["pattern encoded"])
y = list(data_df["intent"])
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.30, shuffle=True, random_state=100, stratify=y
)


Tensor dataset

In [ ]:
from dataloader import get_dataloder


In [ ]:
batch_size = 32
train_data_loader = get_dataloder(X_train, y_train, batch_size=batch_size, shuffle=True)
val_data_loader = get_dataloder(X_valid, y_valid, batch_size=batch_size, shuffle=False)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


In [ ]:
# vectors.shape


In [ ]:
for x, y, l in train_data_loader:
    print(x.shape, l)
    break


classification model

In [ ]:
from model import Classifier, ModelOperations


In [ ]:
hidden_dim = 100
n_layers = 2
bidirectional = True
n_classes = data_df["intent"].nunique()
model = Classifier(
    hidden_dim=hidden_dim,
    n_layers=n_layers,
    n_classes=n_classes,
    bidirectional=bidirectional,
)


In [ ]:
model


In [ ]:
epoch = 100
learning_rate = 1e-3
model_process = ModelOperations(model, device)


In [ ]:
training_accuracy, training_loss, val_accuracy, val_loss, best_epoch = model_process.training(
    train_loader=train_data_loader,
    val_loader=val_data_loader,
    epoch=epoch,
    learning_rate=learning_rate,
)


Plot

In [ ]:
max(val_accuracy)

In [ ]:
np.argmin(val_loss)

In [ ]:
print(training_accuracy[86],training_loss[86])

In [ ]:
print(val_accuracy[86],val_loss[86])

In [ ]:
import matplotlib.pyplot as plt

X = range(1, epoch + 1, 1)


In [ ]:
len(training_accuracy)

In [ ]:
best_epoch

In [ ]:
min(val_loss)

In [ ]:
val_loss[best_epoch]

In [ ]:
val_accuracy[best_epoch]

In [ ]:
training_accuracy[best_epoch]


In [ ]:
training_loss[best_epoch]

In [ ]:
plt.plot(X, training_accuracy, label="train")
plt.plot(X, val_accuracy, label="val")
plt.scatter(best_epoch, training_accuracy[best_epoch], color='red')
plt.scatter(best_epoch, val_accuracy[best_epoch], color='red')
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title('Epoch vs Accuracy')
plt.legend()
plt.show()


In [ ]:
plt.plot(X, training_loss, label="train")
plt.plot(X, val_loss, label="val")
plt.scatter(best_epoch, training_loss[best_epoch], color='red')
plt.scatter(best_epoch, val_loss[best_epoch], color='red')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title('Epoch vs Loss')
plt.legend()
plt.show()


Pred

In [ ]:
ypred, ytrue = model_process.pred(train_data_loader)


In [ ]:
from sklearn.metrics import classification_report

print(classification_report(ytrue, ypred))


In [ ]:
import pickle


In [ ]:
with open("./saved_objects/label_encoding.pkl", "rb") as handle:
    label_encoding = pickle.load(handle)


In [ ]:
label_encoding.inverse_transform([28, 35, 17, 22])


In [ ]:
ypred, ytrue = model_process.pred(val_data_loader)
print(classification_report(ytrue, ypred))


In [ ]:
label_encoding.inverse_transform([10,15,18])


In [ ]:
import torch


In [ ]:
from response_generator import generate_response

max_word = 128

In [ ]:
l = generate_response("how does ml work at your company?")
print(l)

In [ ]:
import time

start = time.time()
l = generate_response("hey")
print(time.time() - start)
print(l)


In [ ]:
l = generate_response("how are you doing")
print(l)


In [ ]:
l = generate_response("what your name")
print(l)


In [ ]:
l = generate_response("what is your name")
print(l)
